# Multi-Cloud Data Governance with Data Virtualization and Cloud Pak for Data

Online version: <url>

### Sample database schema consists of the following tables:

- Big Query - GCP
   - Products: stores a list of scale model cars. 
   - ProductLines: stores a list of product line categories. 
- Db2 for i - On Prems :
   - Customers: stores customer’s data.  
   - Orders: stores sales orders placed by customers.
   - OrderDetails: stores sales order line items for each sales order.
   - Payments: stores payments made by customers based on their accounts.
- Oracle AIX - On prems :  
   - Employees: stores all employee information as well as the organization structure such as who reports to whom.
   - Offices: stores sales office data.

[![Data Fabric RetailOne ](./Pictures/RetailOne-DataFabricDemo.png)](https://github.com/bmarolleau/retailOne/blob/main/README.md "Data Governance and Modernization: Data Fabric, Cloud Pak for Data")

## Table of Contents
1. [Data Virtualization (DV)](./RetailOne-Notebook1-1.ipynb#dv)
2. [Data Governance: Watson Knowledge Catalog (WKC)](./RetailOne-Notebook1-1.ipynb#wkc)

### Data Virtualization <a name="dv"></a>

In [1]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

retail_dv_data_request = {
    'connection_name': """retail-dv""",
    'select_statement': 'SELECT * FROM "RETAIL"."CUSTOMERS"'
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=retail_dv_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo)
data_df_1.head(10)


,CUSTOMERNUMBER,CUSTOMERNAME,CONTACTLASTNAME,CONTACTFIRSTNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,SALESREPEMPLOYEENUMBER,CREDITLIMIT
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",addr,Nantes,None,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166,71800.00
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.00
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370,118200.00
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504,81700.00
5,124,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,None,San Rafael,CA,97562,USA,1165,210500.00
6,125,Havel & Zbyszek Co,Piestrzeniewicz,Zbyszek,(26) 642-7555,ul. Filtrowa 68,None,Warszawa,None,01-012,Poland,None,0.00
7,128,"Blauer See Auto, Co.",Keitel,Roland,+49 69 66 90 2555,Lyonerstr. 34,None,Frankfurt,None,60528,Germany,1504,59700.00
8,129,Mini Wheels Co.,Murphy,Julie,6505555787,5557 North Pendale Street,None,San Francisco,CA,94217,USA,1165,64600.00
9,131,Land of Toys Inc.,Lee,Kwai,2125557818,897 Long Airport Avenue,None,NYC,NY,10022,USA,1323,114900.00


In [2]:
!wget -O db2.ipynb https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVHOL4/main/db2.ipynb
%run db2.ipynb
print('db2.ipynb loaded')

--2022-02-14 09:08:46--  https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVHOL4/main/db2.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168432 (164K) [text/plain]
Saving to: ‘db2.ipynb’

db2.ipynb           100%[===================>] 164.48K  --.-KB/s    in 0.03s   

2022-02-14 09:08:46 (5.27 MB/s) - ‘db2.ipynb’ saved [168432/168432]

Db2 Extensions Loaded.
db2.ipynb loaded


## Connecting to Data Virtualization SQL Engine

In [3]:
# Connect to the Db2 STOCKS database on IBM Cloud Pak for Data
database = 'BIGSQL'
user = 'benoit'
password = 'abc123'
host = 'cpd-zen-40.apps.cp4d.hcs.demo'
port = '31253'

%sql CONNECT TO {database} USER {user} USING {password} HOST {host} PORT {port}

Connection successful.


In [4]:
%sql select TABNAME, TABSCHEMA, OWNER from syscat.tables where TABSCHEMA = 'RETAIL' ;

,TABNAME,TABSCHEMA,OWNER
0,ORDERDETAILS,RETAIL,ADMIN
1,productlines,RETAIL,ADMIN
2,products,RETAIL,ADMIN
3,TOPSALES,RETAIL,ADMIN
4,PAYMENTS,RETAIL,ADMIN
5,CUSTOMERS,RETAIL,ADMIN
6,ORDERS,RETAIL,ADMIN
7,OFFICES,RETAIL,ADMIN
8,EMPLOYEES,RETAIL,ADMIN


## Seeing where your Virtualized Data is coming from

In [5]:
%%sql -a
SELECT TABSCHEMA, TABNAME
  FROM SYSCAT.NICKNAMES
    WHERE TABSCHEMA != 'DVSYS'
    ORDER BY TABSCHEMA, TABNAME

,TABSCHEMA,TABNAME
0,RETAIL,CUSTOMERS
1,RETAIL,EMPLOYEES
2,RETAIL,OFFICES
3,RETAIL,ORDERDETAILS
4,RETAIL,ORDERS
5,RETAIL,PAYMENTS
6,RETAIL,productlines
7,RETAIL,products


In [6]:
%%sql -a
SELECT N.TABSCHEMA AS TABSCHEMA, N.TABNAME AS TABNAME, S.SRCTABNAME AS SRCTABNAME, S.SRCSCHEMA AS SRCSCHEMA, S.SRCTYPE AS TYPE, S.DRIVER AS DRIVER, S.URL AS URL, S.USER AS USER, S.HOSTNAME AS HOSTNAME, S.PORT AS PORT, S.DBNAME AS DBNAME
  FROM SYSCAT.NICKNAMES N, TABLE(
  DVSYS.GET_VT_SOURCES(N.TABSCHEMA, N.TABNAME)) S
  WHERE N.TABSCHEMA != 'DVSYS'

,TABSCHEMA,TABNAME,SRCTABNAME,SRCSCHEMA,TYPE,DRIVER,URL,USER,HOSTNAME,PORT,DBNAME
0,RETAIL,CUSTOMERS,CUSTOMERS,RETAILCRM,DB2,com.ibm.db2.jcc.DB2Driver,jdbc:db2://10.7.19.71:446/O01DD6F4:,benoit,10.7.19.71,446,O01DD6F4
1,RETAIL,PAYMENTS,PAYMENTS,RETAILCRM,DB2,com.ibm.db2.jcc.DB2Driver,jdbc:db2://10.7.19.71:446/O01DD6F4:,benoit,10.7.19.71,446,O01DD6F4
2,RETAIL,ORDERS,ORDERS,RETAILCRM,DB2,com.ibm.db2.jcc.DB2Driver,jdbc:db2://10.7.19.71:446/O01DD6F4:,benoit,10.7.19.71,446,O01DD6F4
3,RETAIL,products,products,"""tests-benoit-marolleau.dataset""",BigQuery,com.ibm.jdbc.googlebigquery.GoogleBigQueryDriver,jdbc:ibm:googlebigquery:AuthenticationMethod=s...,,service-account@tests-benoit-marolleau.iam.gse...,0,tests-benoit-marolleau
4,RETAIL,productlines,productlines,"""tests-benoit-marolleau.dataset""",BigQuery,com.ibm.jdbc.googlebigquery.GoogleBigQueryDriver,jdbc:ibm:googlebigquery:AuthenticationMethod=s...,,service-account@tests-benoit-marolleau.iam.gse...,0,tests-benoit-marolleau
5,RETAIL,ORDERDETAILS,ORDERDETAILS,RETAILCRM,DB2,com.ibm.db2.jcc.DB2Driver,jdbc:db2://10.7.19.71:446/O01DD6F4:,benoit,10.7.19.71,446,O01DD6F4
6,RETAIL,EMPLOYEES,EMPLOYEES,ACMEAIR,Oracle,com.ibm.jdbc.oracle.OracleDriver,jdbc:ibm:oracle://10.3.66.90:1521;SID=acmeair;,acmeair,10.3.66.90,1521,acmeair
7,RETAIL,OFFICES,OFFICES,ACMEAIR,Oracle,com.ibm.jdbc.oracle.OracleDriver,jdbc:ibm:oracle://10.3.66.90:1521;SID=acmeair;,acmeair,10.3.66.90,1521,acmeair


In [7]:
%%sql -a 
select * from table(dvsys.GET_VT_SOURCES('RETAIL', 'PAYMENTS'))

,CID,SRCSCHEMA,SRCTABNAME,SRCTYPE,DRIVER,URL,USER,HOSTNAME,PORT,DBNAME,EXT_CONNID
0,DB210000,RETAILCRM,PAYMENTS,DB2,com.ibm.db2.jcc.DB2Driver,jdbc:db2://10.7.19.71:446/O01DD6F4:,benoit,10.7.19.71,446,O01DD6F4,5d447888-b5b1-49fc-8424-568cff62dbf9


## Joining Virtualized Data

In [8]:
%sql select prod."productName" as "Product Count" from "RETAIL"."products" prod fetch first 10 rows only;

,Product Count
0,1999 Yamaha Speed Boat
1,18th century schooner
2,Pont Yacht
3,The Mayflower
4,The USS Constitution Ship
5,The Schooner Bluenose
6,HMS Bounty
7,The Queen Mary
8,The Titanic
9,1980s Black Hawk Helicopter


In [9]:
sqlin = \
"""
SELECT Db2cus.CUSTOMERNAME as "CustomerName (Db2 for i)", OraEmp.lastname as "LastName (Oracle AIX)" 
FROM "RETAIL"."CUSTOMERS" Db2cus, "RETAIL"."EMPLOYEES" OraEmp 
WHERE Db2cus.SALESREPEMPLOYEENUMBER=OraEmp.employeeNumber;
"""
%sql {sqlin}

,CustomerName (Db2 for i),LastName (Oracle AIX)
0,Mini Gifts Distributors Ltd.,Jennings
1,Mini Wheels Co.,Jennings
2,Technics Stores Inc.,Jennings
3,Corporate Gift Ideas Co.,Jennings
4,The Sharp Gifts Warehouse,Jennings
...,...,...
95,"Vida Sport, Ltd",Gerard
96,CAF Imports,Gerard
97,Precious Collectables,Gerard
98,"Corrida Auto Replicas, Ltd",Gerard


In [10]:
## TOP 10 Sales - Not optmized for DV
# NO CACHE RUN DURATION 4-5 Seconds (remove ORDER BY for immediate result)
sqlin = \
"""
SELECT cus.CUSTOMERNAME as "CustomerName (Db2 for i)", 
       prod."productName" as "Product (GCP Big Query)", 
       pay.paymentDate as "Sales Date (Db2 for i)", 
       emp.lastname as "Sales Rep Name (Oracle)",
       O.CITY as "Sales Rep City(Oracle)",
       det.quantityOrdered as "Qty(Db2 for i)", 
       det.priceEach as "Unit Price",
       det.quantityOrdered * det.priceEach  as "TOTAL"

FROM "RETAIL"."CUSTOMERS" cus, 
     "RETAIL"."products" prod, 
     "RETAIL"."PAYMENTS" pay, 
      "RETAIL"."EMPLOYEES" emp, 
      "RETAIL"."ORDERS" orders, 
      "RETAIL"."ORDERDETAILS" det ,
      "RETAIL"."OFFICES" O

WHERE cus.customerNumber=pay.customerNumber 
     AND cus.SALESREPEMPLOYEENUMBER=emp.employeeNumber 
     AND orders.ordernumber=det.orderNumber 
     AND det.productCode= prod."productCode"
     AND emp.officecode = O.officecode

ORDER BY TOTAL DESC 
 FETCH FIRST 20 ROWS ONLY ;

"""
%sql {sqlin}

,CustomerName (Db2 for i),Product (GCP Big Query),Sales Date (Db2 for i),Sales Rep Name (Oracle),Sales Rep City(Oracle),Qty(Db2 for i),Unit Price,TOTAL
0,"La Corne D'abondance, Co.",2003 Harley-Davidson Eagle Drag Bike,2004-12-04,Bondur,Paris,66.0,174.29,11503.14
1,Mini Gifts Distributors Ltd.,2003 Harley-Davidson Eagle Drag Bike,2004-08-28,Jennings,San Francisco,66.0,174.29,11503.14
2,Mini Gifts Distributors Ltd.,2003 Harley-Davidson Eagle Drag Bike,2005-03-05,Jennings,San Francisco,66.0,174.29,11503.14
3,Baane Mini Imports,2003 Harley-Davidson Eagle Drag Bike,2004-11-28,Jones,London,66.0,174.29,11503.14
4,Baane Mini Imports,2003 Harley-Davidson Eagle Drag Bike,2004-11-04,Jones,London,66.0,174.29,11503.14
...,...,...,...,...,...,...,...,...
15,Signal Gift Stores,2003 Harley-Davidson Eagle Drag Bike,2003-06-06,Thompson,San Francisco,66.0,174.29,11503.14
16,Signal Gift Stores,2003 Harley-Davidson Eagle Drag Bike,2004-12-17,Thompson,San Francisco,66.0,174.29,11503.14
17,Atelier graphique,2003 Harley-Davidson Eagle Drag Bike,2004-12-18,Hernandez,Paris,66.0,174.29,11503.14
18,Atelier graphique,2003 Harley-Davidson Eagle Drag Bike,2003-06-05,Hernandez,Paris,66.0,174.29,11503.14


## Making Complex SQL Simple to Consume

In [11]:
# With DV, you can join 2 virtual tables together or create your own virtual views:     

```sql 
CREATE VIEW "RETAIL".TOPSALES as(
   SELECT cus.CUSTOMERNAME as "CustomerName (Db2 for i)", 
       prod."productName" as "Product (GCP Big Query)", 
       pay.paymentDate as "Sales Date (Db2 for i)", 
       emp.lastname as "Sales Rep Name (Oracle)",
       O.CITY as "Sales Rep City(Oracle)",
       det.quantityOrdered as "Qty(Db2 for i)", 
       det.priceEach as "Unit Price",
       det.quantityOrdered * det.priceEach  as "TOTAL"

FROM "RETAIL"."CUSTOMERS" cus, 
     "RETAIL"."products" prod, 
     "RETAIL"."PAYMENTS" pay, 
      "RETAIL"."EMPLOYEES" emp, 
      "RETAIL"."ORDERS" orders, 
      "RETAIL"."ORDERDETAILS" det ,
      "RETAIL"."OFFICES" O

WHERE cus.customerNumber=pay.customerNumber 
     AND cus.SALESREPEMPLOYEENUMBER=emp.employeeNumber 
     AND orders.ordernumber=det.orderNumber 
     AND det.productCode= prod."productCode"
     AND emp.officecode = O.officecode); 
```

In [10]:
%sql SELECT * from "RETAIL"."TOPSALES" ORDER BY TOTAL DESC FETCH FIRST 200 ROWS ONLY; 

,CustomerName (Db2 for i),Product (GCP Big Query),Sales Date (Db2 for i),Sales Rep Name (Oracle),Sales Rep City(Oracle),Qty(Db2 for i),Unit Price,TOTAL
0,Baane Mini Imports,2003 Harley-Davidson Eagle Drag Bike,2004-11-04,Jones,London,66.0,174.29,11503.14
1,Reims Collectables,2003 Harley-Davidson Eagle Drag Bike,2005-06-09,Bondur,Paris,66.0,174.29,11503.14
2,Reims Collectables,2003 Harley-Davidson Eagle Drag Bike,2003-05-21,Bondur,Paris,66.0,174.29,11503.14
3,Reims Collectables,2003 Harley-Davidson Eagle Drag Bike,2003-07-21,Bondur,Paris,66.0,174.29,11503.14
4,Reims Collectables,2003 Harley-Davidson Eagle Drag Bike,2005-01-10,Bondur,Paris,66.0,174.29,11503.14
...,...,...,...,...,...,...,...,...
195,Signal Gift Stores,2003 Harley-Davidson Eagle Drag Bike,2003-06-06,Thompson,San Francisco,66.0,174.29,11503.14
196,Signal Gift Stores,2003 Harley-Davidson Eagle Drag Bike,2004-12-17,Thompson,San Francisco,66.0,174.29,11503.14
197,Atelier graphique,2003 Harley-Davidson Eagle Drag Bike,2004-12-18,Hernandez,Paris,66.0,174.29,11503.14
198,Atelier graphique,2003 Harley-Davidson Eagle Drag Bike,2003-06-05,Hernandez,Paris,66.0,174.29,11503.14


## Performance and Caching

Example of Query Execution 
- No Cache 290s , 
- With Caching : 100s , to 2.5s

### Data Governance: Watson Knowledge Catalog <a name="wkc"></a>

[![Data Fabric RetailOne ](./Pictures/RetailOne-WKC.png)](https://github.com/bmarolleau/retailOne/blob/main/README.md "Data Governance and Modernization: Data Fabric, Cloud Pak for Data")